In [41]:
from pymongo import MongoClient
import pymongo
import datetime
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torchmetrics.functional import precision_recall
from torchmetrics import Accuracy
from torchmetrics import F1Score
from torchmetrics import Precision
from torchmetrics import Recall

from tqdm import tqdm

In [2]:
# dataloaders
batch_size = 32
device='cuda'

class SentimentLSTM(nn.Module):

    def __init__(self, seq_size, output_size, hidden_dim, n_layers):
        
        super(SentimentLSTM, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        # embedding and LSTM layers
        self.lstm = nn.LSTM(seq_size, hidden_dim, n_layers, batch_first=True, dropout=0.2)
        
        # linear and sigmoid layer
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        batch_size = x.size(0)
        
        lstm_out, hidden = self.lstm(x)
        ## lstm_out.shape [batch, seq_len, hidden_dim]
        ## hidden [batch, hidden_dim]

        # fully connected layer        
        out = self.fc(lstm_out[:, -1, :])
        ## out.shape: [n_layer * n_direction, batch, output_size]

        # sigmoid function
        sig_out = self.sig(out)
        
        # return last sigmoid output and hidden state
        return sig_out, out

In [3]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        # nn.init.uniform_(param.data, -0.1, 0.1)

In [4]:
hidden_dim = 512
n_layers = 4
lr = 0.01
output_size = 1

model = SentimentLSTM(20, output_size, hidden_dim, n_layers)
model.to('cuda')

SentimentLSTM(
  (lstm): LSTM(20, 512, num_layers=4, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [5]:
model.load_state_dict(torch.load("model_weight.pth"))

<All keys matched successfully>

In [6]:
test_data = torch.load('/project/annotation/test_forScore.pt')
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

In [42]:
def predict(model, test_loader, sequence_length=30):
    
    model.eval()
    f1 = F1Score(num_classes=True).to('cuda')
    acc = Accuracy(num_classes=True).to('cuda')
    pre = Precision(num_classes=True).to('cuda')
    rec = Recall(num_classes=True).to('cuda')
    
    batch_size = 32
    acc_list = list()
    pred_list, label_list = list(), list()
    
    
    
    for inputs, labels in test_loader:
        
        inputs, labels = inputs.cuda(), labels.cuda()
        
        output, h = model(inputs)
        
        pred = torch.round(output.squeeze())
        
        #print(f1_score(pred, labels))
        
        accuracy = (pred==labels).sum().item() / 32
        acc_list.append(accuracy)
        
        #pred = list(pred.cpu().detach().numpy())
        #pred_list = pred_list + pred
        #label_list += labels.cpu().numpy()
        
        f1(pred, labels)
        acc(pred, labels)
        pre(pred, labels)
        rec(pred, labels)
        #print('Prediction value, pre-rounding: ', output.item())

        # print custom response based on whether test_review is pos/neg
        
    print(np.mean(acc_list))
    
    print(f1.compute())
    print(acc.compute())
    print(pre.compute())
    print(rec.compute())


In [43]:
predict(model, test_loader)

0.9558445411392406
tensor(0.3918, device='cuda:0')
tensor(0.9562, device='cuda:0')
tensor(0.2659, device='cuda:0')
tensor(0.7441, device='cuda:0')
